# Necessary libraries

In [1]:
#!pip install pmdarima

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
from scipy.interpolate import CubicSpline, InterpolatedUnivariateSpline, Akima1DInterpolator
from datetime import datetime, timedelta
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
#from datetime import timedelta
#import datetime
#import fbprophet
#from matplotlib import pyplot
from pandas import to_datetime
from statsmodels.tsa.arima_model import ARIMA
#from fbprophet import Prophet
#from pmdarima.arima import auto_arima
#import pmdarima
from scipy import interpolate
#from fbprophet import Prophet
from matplotlib import pyplot
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import plotly.graph_objs as go
import sklearn.model_selection
from scipy import stats
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline as CS
from statistics import mean
import math
from pmdarima.arima import auto_arima

2023-09-04 16:06:46.898484: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 16:06:46.920625: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 16:06:46.921140: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 16:06:47.686278: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# df_for=pd.read_csv("6_df_interpolated_for_forecasting_may22.csv")
# df_for=df_for.drop(["Unnamed: 0"],axis=1)
# df_for

In [4]:
df_for=pd.read_csv("6_df_interpolated_for_forecasting_aug10.csv")
#df_for=df_for.drop(["Unnamed: 0"],axis=1)
df_for

,Unnamed: 0,Date,Area,Gender,Race,Ethinicity,Wound
0,0,2016-10-26,4.0000,M,White,Non-Hispanic,1
1,1,2016-10-27,4.0000,M,White,Non-Hispanic,1
2,2,2016-10-28,4.0000,M,White,Non-Hispanic,1
3,3,2016-10-29,4.0000,M,White,Non-Hispanic,1
4,4,2016-10-30,4.0000,M,White,Non-Hispanic,1
...,...,...,...,...,...,...,...
1290115,1290115,2023-01-05,2.7064,F,White,Non-Hispanic,17884
1290116,1290116,2023-01-06,2.7064,F,White,Non-Hispanic,17884
1290117,1290117,2023-01-07,2.7064,F,White,Non-Hispanic,17884
1290118,1290118,2023-01-08,2.7064,F,White,Non-Hispanic,17884


In [6]:
def make_input_for_arima(df):    
    #display(df)
    df=df.reset_index()
    total_n=df.shape[0]
    n=math.ceil(0.15*total_n)
    dff=pd.concat([df, df.tail(n), df.tail(n)]).drop_duplicates(keep=False)
    #display(test)
    #display(dff)
    sample_row=dff.sample(1)
    a=int(sample_row.index.values)
    b=int(sample_row.index.values)+n
    #print(a)
    #print(b)
    test=df[a:b]
    train=pd.concat([df, test, test]).drop_duplicates(keep=False)
    # #display(train)
    return (test,train)

In [20]:
def model_holtwinters(df, test, train):
    #fit1 = sm.tsa.statespace.SARIMAX(train.Area, order=(2, 1, 4),seasonal_order=(0,1,1,7),initialization='approximate_diffuse').fit()
    fit1 = ExponentialSmoothing(np.asarray(train['Area']) ,seasonal_periods=2 ,trend='add', seasonal='add').fit()

    y_hat_avg = test.copy()
    y_hat_avg['Holtwinters'] = fit1.forecast(len(test))
    #print(y_hat_avg['ARIMA'])
    plt.figure(figsize=(16,8))
    #plt.plot( train['Area'], label='Train')
    #plt.plot(test['Area'], label='Test')
    plt.plot(df['Area'],label='Original')
    plt.plot(y_hat_avg['Holtwinters'], label='Forecasted by Holtwinters')
    plt.legend(loc='best')
    #plt.axhline(0)
    plt.grid()
    plt.xlabel('Days')
    plt.ylabel('Wound Area')
    plt.show()




    #Accuracy measure of the model (to be done next)
    forecast=y_hat_avg.Holtwinters
    actual=test.Area
    training_series=train.Area
    testing_series=test.Area
    prediction_series= y_hat_avg.Holtwinters



    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
     #as R-square value does not for forecasted value and actual value because forecasted values are 14 more than actual values, thus I am taking only the forecasted values minusing the last 14 values of the forecasted dataframe.
    # df = df.iloc[:-2]
    #forecast_without_last14=forecast.iloc[:-14]
    #actual_without_nan=actual.dropna()
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    #corr= np.corrcoef(forecast_without_last14,actual_without_nan)[0,1]
    r_square= corr**2
#     mins = np.amin(np.hstack([forecast[:,None], 
#                               actual[:,None]]), axis=1)
#     maxs = np.amax(np.hstack([forecast[:,None], 
#                               actual[:,None]]), axis=1)
#     minmax = 1 - np.mean(mins/maxs)             # minmax
    #acf1 = acf(fc-test)[1]  # ACF1

    n = training_series.shape[0]
    d = np.abs(  np.diff( training_series) ).sum()/(n-1)
    errors = np.abs(testing_series - prediction_series )
    mase=errors.mean()/d

    #print("For Subject ID:", list[i])
#     print("MASE:",mase)
#     print("MAPE:",mape)
#     print("ME:",me)
#     print("MAE:",mae)
#     print("MPE:",mpe)
#     print("RMSE:",rmse)
#     print("CORR:",corr)
#     print("R-SQUARE",r_square)
#     print("MINMAX:",minmax)
    return (rmse, mase, r_square)




In [ ]:
list_wounds=df_for["Wound"].unique()

result_df=pd.DataFrame(index=np.arange(len(list_wounds)))

result_df["Wound"]=np.nan
result_df["Gender"]=np.nan
result_df["Race"]=np.nan
result_df["Ethinicity"]=np.nan
result_df["RMSE"]=np.nan
result_df["MASE"]=np.nan
result_df["R-squared"]=np.nan
    

for i in range(len(list_wounds)):
#     if i>4:
#         break
    print("Wound:",i+1)
    wound=list_wounds[i]
    df_1=df_for.query("Wound==@wound")
    #display(df_1)
    df=df_1[["Date","Area"]]
    test,train=make_input_for_arima(df)
    
    
    result_df["Wound"][i]=int(df_1["Wound"].unique()[0])
    result_df["Gender"][i]=str(df_1["Gender"].unique()[0])
    result_df["Race"][i]=str(df_1["Race"].unique()[0])
    result_df["Ethinicity"][i]=str(df_1["Ethinicity"].unique()[0])
    
    df=df.reset_index()

    if train.shape[0]>1:
        if train.shape[0]==2 or train.shape[0]==3:
            if len(list(set(train.Area.values.tolist())))>1 & (datetime.strptime(train.reset_index()["Date"][train.shape[0]-1],'%Y-%m-%d')-datetime.strptime(train.reset_index()["Date"][0],'%Y-%m-%d')).days>2 and train["Area"].dropna().shape[0]!=0:
            
                rmse, mase, r_square=model_holtwinters(df, test, train)
                result_df["RMSE"][i]=rmse
                result_df["MASE"][i]=mase
                result_df["R-squared"][i]=r_square
                
        if train.shape[0]>3 and train["Area"].dropna().shape[0]!=0:
            
            rmse, mase, r_square=model_holtwinters(df, test, train)
            result_df["RMSE"][i]=rmse
            result_df["MASE"][i]=mase
            result_df["R-squared"][i]=r_square
                
result_df["Wound"]=result_df["Wound"].astype(int)

In [12]:
df

,Date,Area
24968,2019-02-26,1.500000
24969,2019-02-27,1.298571
24970,2019-02-28,1.097143
24971,2019-03-01,0.895714
24972,2019-03-02,0.694286
24973,2019-03-03,0.492857
24974,2019-03-04,0.291429
24975,2019-03-05,0.090000
24976,2019-03-06,0.086786
24977,2019-03-07,0.083571


In [30]:
result_df

,Wound,Gender,Race,Ethinicity,RMSE,MASE,R-squared
0,1,M,White,Non-Hispanic,1.479525,5.066716,0.046923
1,2,M,White,Non-Hispanic,1.014797,4.723285,0.927650
2,3,M,White,Non-Hispanic,0.185481,7.490611,1.000000
3,5,M,White,Non-Hispanic,2.328319,6.992540,0.993764
4,6,M,White,Non-Hispanic,0.872019,15.624282,0.979899
...,...,...,...,...,...,...,...
14913,17877,M,White,Non-Hispanic,2.596683,16.561704,0.887877
14914,17879,F,White,Non-Hispanic,0.093146,5.265298,NaN
14915,17880,F,White,Non-Hispanic,1.795225,5.114129,0.953891
14916,17881,M,White,Non-Hispanic,13.141264,4.433311,1.000000


In [34]:
result_df.to_csv("Holt_winters_result_Aug24.csv")

In [32]:
result_df=result_df.dropna()
result_df=result_df.replace({np.inf:np.nan})
result_df=result_df.dropna()

In [33]:
result_df

,Wound,Gender,Race,Ethinicity,RMSE,MASE,R-squared
0,1,M,White,Non-Hispanic,1.479525,5.066716,0.046923
1,2,M,White,Non-Hispanic,1.014797,4.723285,0.927650
2,3,M,White,Non-Hispanic,0.185481,7.490611,1.000000
3,5,M,White,Non-Hispanic,2.328319,6.992540,0.993764
4,6,M,White,Non-Hispanic,0.872019,15.624282,0.979899
...,...,...,...,...,...,...,...
14911,17875,F,White,Non-Hispanic,0.378447,8.993306,0.996527
14912,17876,F,White,Non-Hispanic,6.212400,9.657690,0.930276
14913,17877,M,White,Non-Hispanic,2.596683,16.561704,0.887877
14915,17880,F,White,Non-Hispanic,1.795225,5.114129,0.953891
